In [1]:
import pandas as pd

In [ ]:
convos = pd.read_csv('/home/jeremy/Projects/toxic_talk/data/conversations.csv')
participants = pd.read_csv

In [12]:
import requests
from bs4 import BeautifulSoup
import csv
import re
from datetime import datetime, timedelta

# Define the base URL for Purdue's academic calendars
base_url = "https://catalog.purdue.edu/preview_program.php?catoid=17&poid=31246"

# Send a GET request to the base URL
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all links to projected academic calendars
calendar_links = []
for link in soup.find_all('a', href=True):
    href = link['href']
    if "Academic Calendar" in link.text:
        print(link.text)
        print(href)
        full_url = "https://catalog.purdue.edu/" + href
        calendar_links.append(full_url)


2024-2025 Academic Calendar
preview_program.php?catoid=17&poid=31355
2025-2026 Academic Calendar
preview_program.php?catoid=17&poid=31829
2026-2027 Academic Calendar
preview_program.php?catoid=17&poid=31759
2027-2028 Academic Calendar
preview_program.php?catoid=17&poid=31760
2028-2029 Academic Calendar
preview_program.php?catoid=17&poid=31761
2029-2030 Academic Calendar
preview_program.php?catoid=17&poid=31762
2030-2031 Academic Calendar
preview_program.php?catoid=17&poid=31763
2031-2032 Academic Calendar
preview_program.php?catoid=17&poid=31764
2032-2033 Academic Calendar
preview_program.php?catoid=17&poid=31765
2033-2034 Academic Calendar
preview_program.php?catoid=17&poid=31766
2034-2035 Academic Calendar
preview_program.php?catoid=17&poid=31767
2035-2036 Academic Calendar
preview_program.php?catoid=17&poid=31350


In [ ]:
def parse_calendar(url):
    import dateutil.parser
    events = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    def get_date_from_string(date_text, year, is_winter = False):
        # Attempt to parse the date string
        has_year = re.search(r'\d{4}', date_text)
        if not has_year:
            if 'January' in date_text and is_winter:
                year = int(year) + 1
            date_text = date_text + ', ' + str(year)
        try:
            parsed_date = dateutil.parser.parse(date_text, fuzzy=True)
            formatted_date = parsed_date.strftime('%m/%d/%Y')
            return formatted_date
        except ValueError:
            return None

    def get_event_from_string(event_text, date_text, year, is_winter = False):
        # Attempt to parse the date string
        formatted_date = get_date_from_string(date_text, year, is_winter)
        if  not formatted_date:
            return None
        event = {
            'Subject': event_text,
            'Start Date': formatted_date,
            'End Date': formatted_date,
            'All Day Event': 'True',
            'Description': '',
            'Location': ''
        }
        return event


    tables = soup.find('div', class_='program_description').find_all('table')
    events = []
    for table in tables:
        year = table.find_all('thead')[0].find_all('h4')[0].get_text(strip=True)
        year = re.search(r'\d{4}', year).group(0)
        year = int(year)
        for row in table.find_all('tr'):
            cols = row.find_all(['td', 'th'])
            event_text = cols[0].get_text(strip=True)
            if not event_text:
                continue
            dates = [x.get_text(strip=True) for x in cols[1:]]
            for i, date_text in enumerate(dates):
                if i > 2:
                    curr_year = year + 1
                else:
                    curr_year = year
                if date_text:
                    event = get_event_from_string(event_text, date_text, curr_year, is_winter = (i == 2))
                    if event is not None:
                        events.append(event)
    return events

# Aggregate all events from the projected calendars
all_events = []
for link in calendar_links:
    events = parse_calendar(link)
    # Check for duplicate events
    for event in events:
        if event not in all_events:
            all_events.append(event)

May 13, 2024
August 19, 2024
December 16, 2024
January 13, 2025
May 19, 2025
June 7
September 13
February 7
June 13
August 2
December 7
January 3
May 3
August 8
December 9-14
May 5-10
August 2
December 14
May 10
August 8
August 3
December 15
May 16-18
August 9
October 7-8
March 17-22
June 19
June 19
November 27-30
December 26-27December 30-31
May 27
May 26
July 4
July 4
September 2
January 20
December 23
December 24 & 25
January 1, 2025
May 19, 2025
August 25, 2025
December 22, 2025
January 12, 2026
May 18, 2026
June 13
September 19
February 6
June 12
August 8
December 13
January 9, 2026
May 2
August 7
Dec. 15-20
May 4-9
August 8
Dec. 20
January 9, 2026
May 9
August 7
August 9
December 21
May 15-17
August 8
October 13-14
March 16-21
June 19
June 19
Nov. 26-29
December 26-31
May 26
May 25
July 4
July 3
September 1
January 19
January 2, 2026
December 24 & 25
January 1, 2026
May 18, 2026
August 24, 2026
December 21, 2026
January 11, 2027
May 17, 2027
June 12
September 18
February 5
June 1

In [ ]:

# Define CSV headers
headers = ['Subject', 'Start Date', 'End Date', 'All Day Event', 'Description', 'Location']

# Write events to CSV
with open('./purdue_academic_calendar.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    for event in all_events:
        if event is not None:
            writer.writerow(event)

print("CSV file 'purdue_academic_calendar.csv' has been created successfully.")


CSV file 'purdue_academic_calendar.csv' has been created successfully.
